In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sb
from xgboost import XGBClassifier,XGBRegressor
import gc
import warnings
warnings.filterwarnings("ignore")

In [ ]:
RAND_VAL=42
num_folds=5


In [ ]:
df_train=pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")
df_test=pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")
df_train.head()

In [ ]:
df_train['typ']=0
df_test['typ']=1
#
df_all=pd.concat([df_train,df_test],axis=0)

In [ ]:
cat_feats = df_all.columns[np.where(df_all.dtypes == "object")[0]].drop('NObeyesdad')
num_feats  = df_all.columns[np.where(df_all.dtypes != "object")[0]].drop(['id','typ'])
##
df_all=pd.get_dummies(df_all,columns=cat_feats)
sc=StandardScaler()
df_all[num_feats]=sc.fit_transform(df_all[num_feats])

In [ ]:
df_train = df_all[df_all.typ==0].reset_index(drop=True)
df_test = df_all[df_all.typ==1].reset_index(drop=True)
##
print(len(df_train))
print(len(df_test))
##
feat_cols=df_train.columns.drop(['id','typ','NObeyesdad'])
X_test=df_test[feat_cols]
##
print(feat_cols)
df_train.head()

In [ ]:
X=df_train[feat_cols]
y=df_train['NObeyesdad']
#
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
xgb_params ={
    'learning_rate': 0.026,
    'n_estimators': 1500,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'early_stopping_rounds':60,
    'max_depth': 19,
    'min_child_weight': 11.7,
    'gamma': 0.40,
    'colsample_bytree': 0.46,
    'random_state':RAND_VAL,
    'subsample': 0.82,
    'reg_alpha': 0.0004,
    'verbosity': 0,
    'reg_lambda': 1.4,
}


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


XGB=XGBClassifier(**xgb_params)
XGB.fit(X_train,y_train, eval_set=[(X_val, y_val)],verbose=200)

y_pred_val = XGB.predict(X_val[feat_cols])
y_pred_val = y_pred_val.astype('int')
acc_val = accuracy_score(y_val, y_pred_val)
print("Accuracy : ",acc_val)

In [ ]:
print(classification_report(y_val, y_pred_val))

In [ ]:
cm = confusion_matrix(y_val, y_pred_val)
cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100
# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sb.heatmap(cm_percentage, annot=True, fmt='.2f', cmap='Blues',
          xticklabels=label_encoder.classes_,yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
y_test = XGB.predict(df_test[feat_cols])
y_test

In [ ]:
y_test=label_encoder.inverse_transform(y_test)
submission = df_test[['id']]
submission['NObeyesdad']=y_test
submission.to_csv("submission.csv", index=False)
submission.head()